In [ ]:
import os
import sys

# Set the environment variable
os.environ["LD_LIBRARY_PATH"] = "/usr/lib/wsl/lib:" + "/usr/local/cuda-11.8/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")


In [2]:
import torch

In [3]:
from projects.mmdet3d_plugin.models.utils.misc import load_model


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!


/home/mist_sophia/miniconda3/envs/omnidrive/lib/python3.9/site-packages/bitsandbytes/cuda_setup/paths.py:93: UserWarning: /home/mist_sophia/miniconda3/envs/omnidrive did not contain libcudart.so as expected! Searching further paths...
  warn(
/home/mist_sophia/miniconda3/envs/omnidrive/lib/python3.9/site-packages/bitsandbytes/cuda_setup/paths.py:27: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/mist_sophia/miniconda3/envs/omnidrive/lib/python3.9/site-packages/cv2/../../lib64')}
  warn(


AttributeError: 'NoneType' object has no attribute 'cuDeviceGetCount'

In [ ]:
llm_path = 'ckpts/pretrain_qformer/'
use_lora = True
frozen = True
lm_head = load_model(llm_path, use_lora, frozen)


You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: /home/mist_sophia/miniconda3/envs/omnidrive/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cget_col_row_stats

In [ ]:
# print all trainable parameters and the sum of parameters
total_params = 0
for name, param in lm_head.named_parameters():
    if param.requires_grad:
        print(name, param.size(), param.device)
        total_params += param.numel()
print('total_params:', total_params)
# calculate the vram usage
print('vram usage:', total_params * 4 / 1024 / 1024, 'MB')

base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.Size([16, 4096]) cpu
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.Size([4096, 16]) cpu
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.Size([16, 4096]) cpu
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.Size([4096, 16]) cpu
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.Size([16, 4096]) cpu
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.Size([4096, 16]) cpu
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight torch.Size([16, 4096]) cpu
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight torch.Size([4096, 16]) cpu
base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight torch.Size([16, 4096]) cpu
base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight torch.Size([4096, 16]) cpu
base_model.model.mod

In [ ]:
# # test inference
# lm_head.eval()
# input_ids = torch.randint(0, 200, (1, 10)) # 50265 is the vocab size， 
# output = lm_head(input_ids)
# print(output)


RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [ ]:
lm_head = lm_head.cuda()
input_ids = torch.randint(0, 200, (1, 10)).cuda()
output = lm_head(input_ids)
print(output)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


CausalLMOutputWithPast(loss=None, logits=tensor([[[-4.1250,  3.6855,  4.9648,  ..., -0.2080, -2.0977, -1.0938],
         [-4.1172,  3.7070,  4.9844,  ..., -0.2076, -2.0918, -1.0859],
         [-4.1875,  3.4023,  5.1406,  ..., -0.1011, -2.0684, -0.9219],
         ...,
         [-3.6445,  4.0117, 10.2031,  ..., -1.4746, -4.4258, -2.0957],
         [-2.8359,  3.2422,  9.6172,  ..., -2.1465, -3.4727, -2.1797],
         [-4.8008,  1.6445,  8.6484,  ..., -2.3789, -5.0234, -3.3418]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>), past_key_values=None, hidden_states=None, attentions=None)


In [ ]:
# simple training test
for it in range(100):
    input_ids = torch.randint(0, 2000, (1, 10)).cuda() #  ,2000 is 
    output = lm_head(input_ids)
    loss = output.logits.mean()
    loss.backward()
    print('iter:', it, 'loss:', loss.item())
    # update lora
    # LlavaLlamaForCausalLM' object has no attribute 'update_lora'
    
    # lm_head.update_lora()
    # print('lora updated')


iter: 0 loss: -2.0485732555389404
iter: 1 loss: -3.696895122528076
iter: 2 loss: -4.588339805603027
iter: 3 loss: -4.525819778442383
iter: 4 loss: -4.069866180419922
iter: 5 loss: -2.3532967567443848
iter: 6 loss: -2.5736355781555176
iter: 7 loss: -3.713627576828003
iter: 8 loss: -0.38266563415527344
iter: 9 loss: -3.4557642936706543
iter: 10 loss: -4.066431999206543
iter: 11 loss: -2.7196075916290283
iter: 12 loss: -2.879365921020508
iter: 13 loss: -2.307417869567871
iter: 14 loss: -3.119807720184326
iter: 15 loss: -0.26283368468284607
iter: 16 loss: -3.4330461025238037
iter: 17 loss: -2.933001756668091
iter: 18 loss: -4.130236625671387
iter: 19 loss: -4.039763927459717
iter: 20 loss: -2.822705030441284
iter: 21 loss: -3.1969642639160156
iter: 22 loss: -4.301186561584473
iter: 23 loss: -4.305310249328613
iter: 24 loss: -3.644876003265381
iter: 25 loss: -3.6827425956726074
iter: 26 loss: -3.849670171737671
iter: 27 loss: -2.803415298461914
iter: 28 loss: -4.035284042358398
iter: 29 los